In [ ]:
from collections import Counter
import pandas as pd
import re
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
import pyLDAvis
from gensim.models import CoherenceModel
import os
from zipfile import ZipFile
import shutil
import spacy
import matplotlib.pyplot as plt
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)
import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
import numpy as np
import json
import glob
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
import spacy
from nltk.corpus import stopwords
import pyLDAvis
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

dat=pd.read_csv('segmentation.txt',sep="\t",names=['id','segmented_text'])
dat=dat['segmented_text'].values.tolist()
dat= ' '.join(map(str, dat))

data1=re.findall(re.compile(r"<phrase_Q=(.+?)>(.+?)<\/phrase>"), dat)
dat=[item[1] for item in data1]

In [ ]:
dat

In [ ]:
!pip install pyLDAvis

In [ ]:
# Topic Modeling

def lemmatization(texts, allowed_postags=["NOUN", "ADJ", "VERB", "ADV"]):
    nlp = spacy.load("en_core_web_sm", disable=["parser", "ner"])
    texts_out = []
    for text in texts:
        doc = nlp(text)
        new_text = []
        for token in doc:
            if token.pos_ in allowed_postags:
                new_text.append(token.lemma_)
        final = " ".join(new_text)
        texts_out.append(final)
    return (texts_out)

lemmatized_txt = lemmatization(dat)
print (lemmatized_txt)

['region', 'bovine', 'chromosome', 'beef', 'genome', '', 'microsatellite', 'dna', 'autosomal', 'bovine', 'chromosome', '', 'shear', '', 'postmortem', 'muscle', '', 'bull', 'taurus', '', 'beef', '', 'cM', '% confidence interval', 'cM', '', 'group', 'shear', '', 'postmortem', '', 'phenotypic', 'standard deviation', 'phenotypic', 'variance', 'group', 'environmental factor', 'mask', '', '', 'environmental factor', 'genetic', 'future research', '', '', 'group', 'lead', 'meat', 'postmortem', 'live', 'animal', 'management', 'bovine', 'genome', 'trait', '', 'trait', '', 'genotype', 'microsatellite', '', '', 'taurus', 'taurus', '', 'birth weight', '', 'wean', '', 'hot', 'dp', 'fat', '', '', 'muscle', 'area', '', 'rib', 'bone', 'rib', 'fat', 'rib', 'muscle', 'retail', 'fat', 'trim', 'bone', '', 'retail', 'fat', 'bone', 'datum', 'generate', 'f', 'cM', 'chromosome', 'regression', '', 'conditional probability', '', 'compelling evidence', '', '', 'dp', 'chromosome', '', 'genome', '', '', 'result', '

In [ ]:
import gensim.models

def gen_words(texts):
    final = []
    for text in texts:
        new = gensim.utils.simple_preprocess(text, deacc=True)
        final.append(new)
    return (final)

data_words = gen_words(lemmatized_txt)

print (data_words)

id2word = corpora.Dictionary(data_words)
corpus = []
for text in data_words:
    new = id2word.doc2bow(text)
    corpus.append(new)

word = id2word[[0][:1][0]]
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,id2word=id2word,num_topics=5,random_state=100,update_every=1,chunksize=100,passes=10,alpha="auto")
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word, mds="mmds", R=30)
pyLDAvis.save_html(vis, '2topics.html')


# Perplexity and Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=dat, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)
print('\nPerplexity: ', lda_model.log_perplexity(corpus))

[['region'], ['bovine'], ['chromosome'], ['beef'], ['genome'], [], ['microsatellite'], ['dna'], ['autosomal'], ['bovine'], ['chromosome'], [], ['shear'], [], ['postmortem'], ['muscle'], [], ['bull'], ['taurus'], [], ['beef'], [], ['cm'], ['confidence', 'interval'], ['cm'], [], ['group'], ['shear'], [], ['postmortem'], [], ['phenotypic'], ['standard', 'deviation'], ['phenotypic'], ['variance'], ['group'], ['environmental', 'factor'], ['mask'], [], [], ['environmental', 'factor'], ['genetic'], ['future', 'research'], [], [], ['group'], ['lead'], ['meat'], ['postmortem'], ['live'], ['animal'], ['management'], ['bovine'], ['genome'], ['trait'], [], ['trait'], [], ['genotype'], ['microsatellite'], [], [], ['taurus'], ['taurus'], [], ['birth', 'weight'], [], ['wean'], [], ['hot'], ['dp'], ['fat'], [], [], ['muscle'], ['area'], [], ['rib'], ['bone'], ['rib'], ['fat'], ['rib'], ['muscle'], ['retail'], ['fat'], ['trim'], ['bone'], [], ['retail'], ['fat'], ['bone'], ['datum'], ['generate'], [], 

/usr/local/lib/python3.8/dist-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  term_ix = topic_terms.unique()
/usr/local/lib/python3.8/dist-packages/gensim/topic_coherence/direct_confirmation_measure.py:193: RuntimeWarning: invalid value encountered in true_divide
  numerator = (co_occur_count / num_docs) + EPSILON
/usr/local/lib/python3.8/dist-packages/gensim/topic_coherence/direct_confirmation_measure.py:194: RuntimeWarning: invalid value encountered in true_divide
  denominator = (w_prime_count / num_docs) * (w_star_count / num_docs)
/usr/local/lib/python3.8/dist-packages/gensim/topic_coherence/direct_confirmation_measure.py:189: RuntimeWarning: invalid value encountered in true_divide
  co_doc_prob = co_occur_count / num_docs



Coherence Score:  nan

Perplexity:  -8.859500571589033


In [ ]:
!pip install pyldavis==2.1.2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install pyLDAvis

import pyLDAvis
#import pyLDAvis.gensim_models
import pyLDAvis.gensim

lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,id2word=id2word,num_topics=2,random_state=100,update_every=1,chunksize=100,passes=10,alpha="auto")

vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word, mds="mmds", R=30)
pyLDAvis.save_html(vis, '2topics.html')

lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,id2word=id2word,num_topics=3,random_state=100,update_every=1,chunksize=100,passes=10,alpha="auto")

vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word, mds="mmds", R=30)
pyLDAvis.save_html(vis, '3topics.html')
print('\nCoherence Score: ', coherence_lda)
print('\nPerplexity: ', lda_model.log_perplexity(corpus))

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


KeyboardInterrupt: ignored

In [ ]:
from spacy.tokens import Doc
topics = lambda doc: lda_model[gensim_dict.doc2bow([token.lemma_ for token in doc])][0]
Doc.set_extension('topics', getter=topics)

lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                          id2word=id2word,
                                          num_topics=4,
                                          random_state=100,
                                          update_every=1,
                                          chunksize=100,
                                          passes=10,
                                          alpha="auto")
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word, mds="mmds", R=30)
pyLDAvis.save_html(vis, '4topics.html')
print('\nCoherence Score: ', coherence_lda)
print('\nPerplexity: ', lda_model.log_perplexity(corpus))

lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                          id2word=id2word,
                                          num_topics=5,
                                          random_state=100,
                                          update_every=1,
                                          chunksize=100,
                                          passes=10,
                                          alpha="auto")
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word, mds="mmds", R=30)
pyLDAvis.save_html(vis, '5topics.html')
print('\nCoherence Score: ', coherence_lda)
print('\nPerplexity: ', lda_model.log_perplexity(corpus))

/usr/local/lib/python3.8/dist-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  term_ix = topic_terms.unique()
/usr/local/lib/python3.8/dist-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  term_ix = topic_terms.unique()
